# Stream input data

In [ ]:
import warnings

import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt

import tams

warnings.filterwarnings("ignore", category=cartopy.io.DownloadWarning)

## GPM precipitation and brightness temperature

With the help of [earthaccess](https://earthaccess.readthedocs.io/), we can stream these data without downloading the files to disk.
* {func}`tams.data.get_imerg`
* {func}`tams.data.get_mergir`

In [ ]:
%%time

pr = tams.data.get_imerg("2024-06-01 02:30")["pr"]
pr

In [ ]:
%%time

tb = tams.data.get_mergir(pr.time.item())["tb"]
tb

In [ ]:
fig = plt.figure(figsize=(13, 5))
ax = fig.add_subplot(projection=ccrs.PlateCarree())

ax.coastlines(color="magenta")

tb.plot(x="lon", cmap="gist_gray_r", robust=True, ax=ax)
pr.plot(x="lon", norm=mpl.colors.LogNorm(0.01, pr.quantile(0.99)), alpha=0.85, ax=ax);

The white dots in the brightness temperature field above represent missing data.
For most CE identification methods, this will influence results.
Since they are mostly scattered points, not large regions,
a reasonable way to fill in the missing data is a nearest-neighbor interpolation.

In [ ]:
%%time

kws = dict(method="nearest", fill_value="extrapolate", assume_sorted=True)
tb_ = tb.interpolate_na("lat", **kws).interpolate_na("lon", **kws)

print(f"{tb.isnull().sum().item() / tb.size:.3%} -> {tb_.isnull().sum().item() / tb_.size:.3%} null")

In [ ]:
box = dict(lon=slice(-115, -80), lat=slice(-53, -20))
tb_.sel(**box).plot(cmap="gist_gray_r", robust=True)
plt.gca().set_aspect("equal", "box")